# 🧪 Практическое занятие: Логистическая регрессия для анализа социально-экономической ситуации в регионах РФ

## Учебный вопрос 1: Формирование набора данных

In [1]:
import pandas as pd

# Загрузка данных
df = pd.read_csv("RussianRegionsProblem.csv", delimiter=";", on_bad_lines='skip')
print('Первые строки датафрейма:')
print(df.head())

Первые строки датафрейма:
                  Регион  \
0         Алтайский край   
1       Амурская область   
2  Архангельская область   
3   Астраханская область   
4   Белгородская область   

  Отношение медианных доходов к стоимости фиксированного набора товаров и услуг  \
0                                               1,47                              
1                                               2,06                              
2                                               1,96                              
3                                               1,71                              
4                                               2,26                              

  Доля населения за чертой бедности в 2024 г., %  \
0                                           12,1   
1                                            9,5   
2                                            8,1   
3                                           10,7   
4                                            4

### 📌 Задание 1.0: Предварительная очистка — удаление полностью пустых регионов

In [2]:
# Замена 'н/д' и '-' на NaN
df.replace({"н/д": None, "-": None, "Н/Д": None}, inplace=True)

# Удаление строк, где все показатели (кроме названия региона) равны NaN
df_cleaned = df.dropna(subset=df.columns[1:], how='all')

print('\nЧисло регионов после удаления полностью пустых строк:', len(df_cleaned))
print('Примеры оставшихся регионов:')
print(df_cleaned[['Регион'] + list(df.columns[1:4])].head())


Число регионов после удаления полностью пустых строк: 85
Примеры оставшихся регионов:
                  Регион  \
0         Алтайский край   
1       Амурская область   
2  Архангельская область   
3   Астраханская область   
4   Белгородская область   

  Отношение медианных доходов к стоимости фиксированного набора товаров и услуг  \
0                                               1,47                              
1                                               2,06                              
2                                               1,96                              
3                                               1,71                              
4                                               2,26                              

  Доля населения за чертой бедности в 2024 г., %  \
0                                           12,1   
1                                            9,5   
2                                            8,1   
3                                    

### 📌 Задание 1.1: Преобразуйте числовые столбцы в тип `float`

In [3]:
# Список всех числовых столбцов, кроме 'Регион'
numeric_cols = df_cleaned.columns[1:]

# Замена запятых на точки и преобразование в float
for col in numeric_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col].astype(str).str.replace(',', '.'), errors='coerce')

print('\nТипы данных после преобразования:')
print(df_cleaned.dtypes)


Типы данных после преобразования:
Регион                                                                            object
Отношение медианных доходов к стоимости фиксированного набора товаров и услуг    float64
Доля населения за чертой бедности в 2024 г., %                                   float64
Рейтинговый балл качества жизни (2024)                                           float64
Изменение числа работающих за 2023 год, %                                        float64
Доля безработных в IV квартале 2023 года, %                                      float64
Рейтинговый балл по научно-техническому прогрессу                                float64
Рейтинговый балл по ЗОЖ                                                          float64
Интегральный рейтинг по социально-экономическому развитию, баллы                 float64
Место в итоговом рейтинге                                                        float64
dtype: object


C:\Users\Admin\AppData\Local\Temp\ipykernel_14788\2693926024.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col] = pd.to_numeric(df_cleaned[col].astype(str).str.replace(',', '.'), errors='coerce')
C:\Users\Admin\AppData\Local\Temp\ipykernel_14788\2693926024.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col] = pd.to_numeric(df_cleaned[col].astype(str).str.replace(',', '.'), errors='coerce')
C:\Users\Admin\AppData\Local\Temp\ipykernel_14788\2693926024.py:6: SettingWithCopyWarn

### 📌 Задание 1.2: Обработайте пропуски в колонке `Место в итоговом рейтинге`

In [4]:
# Очистка колонки 'Место в итоговом рейтинге'
df_cleaned['Место в итоговом рейтинге'] = pd.to_numeric(
    df_cleaned['Место в итоговом рейтинге'], errors='coerce'
)

# Удаление строк с пустым местом
df_ready = df_cleaned.dropna(subset=['Место в итоговом рейтинге']).copy()

print('\nЧисло регионов после удаления строк с неизвестным местом:', len(df_ready))


Число регионов после удаления строк с неизвестным местом: 75


C:\Users\Admin\AppData\Local\Temp\ipykernel_14788\2811645897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Место в итоговом рейтинге'] = pd.to_numeric(


### 📌 Задание 1.3: Создайте целевую переменную `target`

In [5]:
# Добавление целевой переменной
df_ready['target'] = df_ready['Место в итоговом рейтинге'].apply(
    lambda x: 1 if x <= 30 else (0 if x > 60 else None)
)

# Удаление строк с неопределённым классом
df_final = df_ready.dropna(subset=['target']).copy()
df_final['target'] = df_final['target'].astype(int)

print('\nРаспределение классов:')
print(df_final['target'].value_counts())

print('\nПервые строки финального датафрейма:')
print(df_final[['Регион', 'Место в итоговом рейтинге', 'target']].head())


Распределение классов:
target
1    26
0    20
Name: count, dtype: int64

Первые строки финального датафрейма:
                          Регион  Место в итоговом рейтинге  target
0                 Алтайский край                       61.0       0
1               Амурская область                       64.0       0
4           Белгородская область                       16.0       1
7          Волгоградская область                       29.0       1
11  Еврейская автономная область                       83.0       0


### 📌 Задание 1.4: Выберите признаки и разделите данные на обучающую и тестовую выборки

In [6]:
from sklearn.model_selection import train_test_split

# Признаки
X = df_final.iloc[:, 1:-1]  # все колонки, кроме 'Регион' и 'target'
y = df_final['target']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'\nОбучающая выборка: {X_train.shape}')
print(f'Тестовая выборка: {X_test.shape}')


Обучающая выборка: (36, 9)
Тестовая выборка: (10, 9)
